In [756]:
import os
import pandas as pd
import numpy as np

In [613]:
try:
    print("Repo path:", repo_path)
except:
    os.chdir("../")
    repo_path = os.getcwd()
    print("Repo path:", repo_path)

Repo path: /Users/tx44ru/Projects/Kaggle/DisasterTweets


In [708]:
data_path = "data"
use_dir = "use"

os.chdir(repo_path)

In [615]:
os.listdir(data_path)

['test.csv', 'train.csv', 'sample_submission.csv']

In [616]:
df = pd.read_csv(os.path.join(data_path, "train.csv"))
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Text data cleanning

In [104]:
import string
import nltk
import re

ps = nltk.PorterStemmer()

def txt_cleanning(text):
    """ cleanning:
            lowercase
            remove numbers
            remove punctuation
            stem words
    """
    text  = "".join([char.lower() for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens]  # remove stopwords and stemming
    text = " ".join([str(elem) for elem in text])
    return text

df['text'] = df['text'].apply(lambda x: txt_cleanning(x))
df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,our deed are the reason of thi earthquak may a...,1
1,4,NaN,NaN,forest fire near la rong sask canada,1
2,5,NaN,NaN,all resid ask to shelter in place are be notif...,1
3,6,NaN,NaN,peopl receiv wildfir evacu order in california,1
4,7,NaN,NaN,just got sent thi photo from rubi alaska as sm...,1
5,8,NaN,NaN,rockyfir updat california hwi close in both di...,1
6,10,NaN,NaN,flood disast heavi rain caus flash flood of st...,1
7,13,NaN,NaN,im on top of the hill and i can see a fire in ...,1
8,14,NaN,NaN,there an emerg evacu happen now in the build a...,1
9,15,NaN,NaN,im afraid that the tornado is come to our area,1


### Train/Test Split

In [617]:
from sklearn.model_selection import train_test_split

In [618]:
X_df, y_df = df["text"], df["target"]

In [619]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.25, random_state=0)

In [620]:
print("train:", X_train.shape)
print("test:", X_test.shape)

train: (5709,)
test: (1904,)


### USE

In [731]:
import tensorflow_hub as hub
import tensorflow as tf
from tqdm import tqdm

try:
    print("USE module:\n", use_module)
except:
    use_module = hub.load(use_dir)
    print("USE module:\n", use_module)

USE module:
 <tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7f835cf25bd0>


In [712]:
def get_use(text, use_module):
    text = [text]
    emb = use_module.signatures["response_encoder"](
        input=tf.constant(text),
        context=tf.constant(text))["outputs"].numpy()
    return emb

In [725]:
test = "This is a test"
get_use(test, use_module).squeeze().shape

(512,)

In [733]:
X_train_use = []

for text in tqdm(X_train):
    X_train_use.append(get_use(text, use_module).squeeze())

100%|██████████| 5709/5709 [03:14<00:00, 29.40it/s]


In [745]:
X_test_use = []

for text in tqdm(X_test):
    X_test_use.append(get_use(text, use_module).squeeze())

100%|██████████| 1904/1904 [01:04<00:00, 29.55it/s]


In [742]:
X_train_use[0].shape

(512,)

In [ ]:
X_train_use = []

for text in tqdm(X_train):
    X_train_use.append(get_use(text, use_module).squeeze())

### TF-IDF

In [680]:
# Create the vectorizer method
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer(min_df=3, max_df=0.8, stop_words="english")

In [681]:
tfidf_vec.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.8, max_features=None,
                min_df=3, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [682]:
X_train_tfidf = tfidf_vec.transform(X_train).toarray()
X_test_tfidf = tfidf_vec.transform(X_test).toarray()

In [683]:
X_train_tfidf.shape

(5709, 3372)

## Naive Bayes classifier

In [746]:
X_train_input = X_train_use
X_test_input = X_test_use

In [744]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train_input, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [747]:
# Print out the model's accuracy
nb.score(X_test_input, y_test)

0.7478991596638656

## Deep Net classifier

In [777]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

torch.manual_seed(1)

In [778]:
class SimpleNet(nn.Module):
    
    def __init__(self, input_size):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, round(input_size / 2))
        self.fc2 = nn.Linear(round(input_size / 2), 100)
        self.fc3 = nn.Linear(100, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x
    
    def predict(self, x):
        #Apply sigmoid to output.
        pred = self.forward(x)
        return pred.round()

In [930]:
#Deeper NN
class SimpleNet_v2(nn.Module):
    
    def __init__(self, input_size):
        super(SimpleNet_v2, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.dropout1 = nn.Dropout(p = 0.6)
        self.fc2 = nn.Linear(512, 512)
        self.dropout2 = nn.Dropout(p = 0.3)
        self.fc3 = nn.Linear(512, 32)
        self.fc4 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = self.dropout1(F.relu(self.fc1(x)))
        x = self.dropout2(F.relu(self.fc2(x)))
        x = F.relu(self.fc3(x))
        out = torch.sigmoid(self.fc4(x))
        return out
    
    def predict(self, x):
        #Apply sigmoid to output.
        pred = self.forward(x)
        return pred.round()


In [931]:
X_train_input = X_train_use
X_test_input = X_test_use

In [932]:
def get_dimension(inp):
    if isinstance(inp, list):
        return len(inp[0])
    if isinstance(inp, np.ndarray):
        return inp.shape[1]
    return -1

In [933]:
simple_classifier = SimpleNet_v2(get_dimension(X_train_input))

In [934]:
def save_model(epoch, model, optimizer, scheduler, name):
    train_state = {    
    'model' : model,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'scheduler': scheduler.state_dict()
    }
    torch.save(train_state, name)
    pass

In [935]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(simple_classifier.parameters(), lr=0.001, weight_decay = 0.007);
lr_scheduler = ReduceLROnPlateau(optimizer, 'max', factor = 0.1, patience = 5, verbose=True)

# train_loader, test_loader = create_dataloaders(balanced_train_images, balanced_train_labels, pr_test_img, pr_test_labels)
best_accuracy = 0.0

### Training

In [936]:
%%time
EARLY_STOP = 0
EPOCH = 50
batchsize = 128
model_name = 'SimpleNet_use_3layers.pt'

for epoch in range(EPOCH):  # loop over the dataset multiple times
    if EARLY_STOP >= 21:
        print("STOPPED EARLY!!")
        break
    simple_classifier.train()
    running_loss = 0.0
    for i in range(0, len(y_train), batchsize):
        
        inputs = torch.tensor(X_train_input[i:i+batchsize]).float()
        lbs = torch.tensor(y_train[i:i+batchsize].values).float()

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = simple_classifier(inputs)
        loss = criterion(outputs, lbs)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if (i/batchsize) % 30 == 29:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, (i/batchsize) + 1, running_loss / 1000))
            running_loss = 0.0

    simple_classifier.eval()
    correct = 0
    total = 0
    for j in range(0, len(y_test), batchsize):
        labels = torch.tensor(y_test[j:j+batchsize].values)
        outputs = simple_classifier.predict(torch.tensor(X_test_input[j:j+batchsize]).float())
        total += labels.shape[0]
        correct += outputs.squeeze().eq(labels).sum().item()
    cur_accuracy = correct / total
    print('Accuracy on EPOCH %d test images: %d %%' % (epoch+1, 100 * cur_accuracy))   
    lr_scheduler.step(cur_accuracy)
    if cur_accuracy > best_accuracy:
            best_accuracy = cur_accuracy
            print("Saving current model!!!")
            print("Detailed Accuracy: %f" %(best_accuracy))
            save_model(epoch, simple_classifier, optimizer, lr_scheduler, model_name)
            EARLY_STOP = 0
    EARLY_STOP += 1
print('Finished Training')

[1,    30] loss: 0.021
Accuracy on EPOCH 1 test images: 58 %
Saving current model!!!
Detailed Accuracy: 0.581408
[2,    30] loss: 0.019
Accuracy on EPOCH 2 test images: 81 %
Saving current model!!!
Detailed Accuracy: 0.815126
[3,    30] loss: 0.015
Accuracy on EPOCH 3 test images: 80 %
[4,    30] loss: 0.013
Accuracy on EPOCH 4 test images: 81 %
[5,    30] loss: 0.013
Accuracy on EPOCH 5 test images: 81 %
Saving current model!!!
Detailed Accuracy: 0.819328
[6,    30] loss: 0.013
Accuracy on EPOCH 6 test images: 82 %
Saving current model!!!
Detailed Accuracy: 0.820903
[7,    30] loss: 0.013
Accuracy on EPOCH 7 test images: 81 %
[8,    30] loss: 0.013
Accuracy on EPOCH 8 test images: 81 %
[9,    30] loss: 0.012
Accuracy on EPOCH 9 test images: 81 %
[10,    30] loss: 0.013
Accuracy on EPOCH 10 test images: 82 %
[11,    30] loss: 0.013
Accuracy on EPOCH 11 test images: 81 %
[12,    30] loss: 0.013
Accuracy on EPOCH 12 test images: 82 %
Epoch    12: reducing learning rate of group 0 to 1.00

## LSTM

### Load best model

In [937]:
# give best_classifier the class of the model you want to load
best_classifier = simple_classifier

best_classifier.load_state_dict(torch.load('SimpleNet_use_3layers.pt')['state_dict'])

<All keys matched successfully>

In [938]:
best_classifier.eval()
preds = best_classifier.predict(torch.tensor(X_test_input).float()).detach().numpy()
print("Accuracy - Test:")
print((preds.squeeze() == y_test).sum().item() / y_test.shape[0])

Accuracy - Test:
0.8266806722689075


In [939]:
best_classifier.eval()
preds = best_classifier.predict(torch.tensor(X_train_input).float()).detach().numpy()
print("Accuracy - Train:")
print((preds.squeeze() == y_train).sum().item() / y_train.shape[0])

Accuracy - Train:
0.8421790155894202


### Submit

In [940]:
df_test = pd.read_csv(os.path.join(data_path, "test.csv"))
# df_test["text"] = df_test["text"].apply(lambda x: txt_cleanning(x))
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [941]:
submit_X, submit_y = df_test["text"], df_test["text"]

In [942]:
submit_X_tfidf = tfidf_vec.transform(submit_X).toarray()

In [943]:
submit_X_use = []

for text in tqdm(submit_X):
    submit_X_use.append(get_use(text, use_module).squeeze())

100%|██████████| 3263/3263 [01:52<00:00, 29.04it/s]


In [944]:
submit_X_inputs = submit_X_use

In [945]:
submit_preds = best_classifier.predict(torch.tensor(submit_X_inputs).float()).detach().numpy().astype(int)

In [946]:
to_submit_df = pd.DataFrame(submit_preds, columns=["target"])

to_submit_df = pd.concat((df_test["id"], to_submit_df), axis = 1)

to_submit_df.to_csv("submission.csv", index=False)

In [947]:
!kaggle competitions submit -c nlp-getting-started  -f submission.csv -m "Test"

100%|██████████████████████████████████████| 22.2k/22.2k [00:02<00:00, 7.95kB/s]
403 - Your team has used its submission allowance (5 of 5). This resets at midnight UTC (3.2 hours from now).
